In [1]:
# !conda env create -f /kaggle/input/condaenv/environment.yml
# !conda init
# !conda activate pestvision
# !wget -P /kaggle/input/ https://storage.googleapis.com/npss-pestvision-data/pestvision_data.zip
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 34.1 MB/s eta 0:00:00


In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'
import numpy as np
import torch
import yaml

In [3]:
root = '/kaggle/input/pestvisiondata/pestvision_data/'
dset_paths = ['synthetic_data/DeepImageBlendingData/run3/RiceLeafs/images/train',
             'synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/images/train',
             'synthetic_data/Libcom_HarmonizationData_PCTNet/run1/RiceLeafs/images/train',
             'synthetic_data/Libcom_HarmonizationData_PCTNet/run1/paddy-disease-classification/images/train']
dset_paths = [os.path.join(root,i) for i in dset_paths]
val_paths = ["/".join(i.split('/')[:-1]+['val']) for i in dset_paths]

In [4]:
# object detection code base with abstract class

import os
from abc import ABC, abstractmethod
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback


class AbstractPestDetection(ABC):
    """
    Abstract class for pest detection
    """

    def __init__(self, device):
        self.device = device

    @abstractmethod
    def load_model(self, model_path):
        pass

    @abstractmethod
    def train(self):
        pass

    # TODO: implement evaluate method
    # @abstractmethod
    # def evaluate(self):
    #     pass


class PestDetection_yolov8(AbstractPestDetection):
    """
    YOLOv8 model for pest detection
    """

    def __init__(self, device):
        super().__init__(device)
        self.model = None

    def load_model(self, model_path):
        """
        Load the YOLOv8 model

        Parameters:

         model_path (str): path to the model checkpoint
        """

        self.model = YOLO(model_path)

    def train(self,**kwargs):

        results = self.model.train(**kwargs)
        self.model.val()

        return results

    # TODO: implment evaluate method
    # def evaluate(self):
    #     pass


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [5]:
def make_dsetconfig(train,val):

    d_config = {
        "train": train,
        "val": val,

        "nc": 102, # number of classes
        "names": [str(i) for i in range(1,103)] 
    }
    filename = 'data.yaml'
    with open(filename, 'w') as file:
        yaml.dump(d_config, file, default_flow_style=False)
    return filename

In [6]:
def train(checkpoint,train,val):
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("CUDA is available. Using GPU.")
    else:
        device = torch.device('cpu')
        print("CUDA is not available. Using CPU.")
    model = PestDetection_yolov8(device)
    model.load_model(checkpoint)
#     wandb_kwargs = {"project": None, "entity": None, "group": None, "job_type": None}
    results = model.train(data=make_dsetconfig(train,val), epochs=50, imgsz=512, save_period=10,batch=-1, plots=True)

In [7]:
train('/kaggle/input/yolo/other/exp1/6/last-5.pt', dset_paths, val_paths[1])

CUDA is available. Using GPU.
Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolo/other/exp1/6/last-5.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=512, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 63.5MB/s]
2024-05-26 05:35:34,998	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-26 05:35:36,307	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3341330 parameters, 0 gradients, 9.6 GFLOPs


val: Scanning /kaggle/input/pestvisiondata/pestvision_data/synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/labels/val... 2073 images, 233 backgrounds, 0 corrupt: 100%|██████████| 2306/2306 [00:04<00:00, 554.69it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/pestvisiondata/pestvision_data/synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/labels is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:22<00:00,  2.04it/s]


                   all       2306      11390      0.294      0.263      0.228      0.127
                     1       2306         77      0.334      0.519      0.365      0.174
                     2       2306         83       0.21      0.106     0.0791      0.056
                     3       2306         15          0          0      0.021     0.0124
                     4       2306        110      0.389      0.391      0.319      0.153
                     5       2306         47      0.287      0.468      0.264      0.166
                     6       2306         62      0.293      0.113      0.134     0.0668
                     7       2306         21          0          0     0.0309     0.0233
                     8       2306         74       0.11     0.0541     0.0615     0.0317
                     9       2306         48      0.241      0.104      0.147     0.0702
                    10       2306         33      0.224     0.0606     0.0437     0.0275
                    1